In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
print(train.info())
print('*'*40)
print(test.info())

In [ ]:
train.head(10)

The columns with any null values at all.

In [ ]:
nullCols = train.columns[train.isna().any()].tolist()
print(nullCols)



Number of null values in these columns. 

In [ ]:
train[nullCols].isnull().sum()

**Analysing the null columns one by one.**

1.**Lot Frontage** - Handled later... <br>

2.**ALLEY**

In [ ]:
train["Alley"].value_counts()

The NA values in the Alley mean that there is no alley access,i.e, they are not actually null values.

In [ ]:
train["Alley"] = train["Alley"].fillna(value = 'NoAlley')
test["Alley"] = test["Alley"].fillna(value = "NoAlley")

3.**Masonry Veneer Type**

In [ ]:
train["MasVnrType"].value_counts()

We can replace the null values of the MasVnrType with the respective mode values from the train set, which correspond to those types of properties using the correlation matrix.

In [ ]:
train[train["MasVnrType"].isnull()]

In [ ]:
pd.set_option('display.max_rows', 1000)

In [ ]:
#train.groupby("MSZoning").mode()
from scipy import stats
newDF = train[train["MasVnrType"].notnull()].groupby(["MSZoning","MSSubClass","Alley","LotShape","LandContour","LotConfig"]).agg(lambda x: stats.mode(x)[0][0])
display(newDF['MasVnrType'].loc[["RL"],:])

Thus, appropriate analysis shows that for ID = 235 & 1279, the feature MasVnrType would be filled with "BrkFace" and with "None" for the rest.

In [ ]:
train.loc[train["Id"]==235,"MasVnrType"] = "BrkFace"
train.loc[train["Id"]==1279,"MasVnrType"] = "BrkFace"

In [ ]:
train["MasVnrType"] = train["MasVnrType"].fillna("None")

4.**MasVnrArea**

In [ ]:
train[train["MasVnrType"]=="None"]["MasVnrArea"]

In [ ]:
train.loc[train["MasVnrType"]=="None","MasVnrArea"] = 0

In [ ]:
train[train["MasVnrArea"].isnull()]

In [ ]:
train[train["MasVnrType"]=="BrkFace"].mean()["MasVnrArea"]

We fill in the MasVnrArea with the mean value, i.e, 259.

In [ ]:
train.loc[train["MasVnrArea"].isnull(),"MasVnrArea"] = 259

5.**BsmtQual, Bsmt Condition, Bsmt Exposure, Bsmt FinType1, Bsmt FinType2**

In [ ]:
train.loc[train["BsmtQual"].isnull(),["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]] = "NoBasement"
test.loc[test["BsmtFinType1"].isnull(),["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]] = "NoBasement"

In [ ]:
test[test["BsmtCond"].isnull()]

In [ ]:
test[test["BsmtCond"].notnull()].groupby(["BsmtQual","BsmtExposure","BsmtFinType1","BsmtFinType2"]).agg(lambda x: stats.mode(x)[0][0])["BsmtCond"]

In [ ]:
test["BsmtCond"] = test["BsmtCond"].fillna("TA")

In [ ]:
test[test["BsmtQual"].isnull()]

In [ ]:
test[test["BsmtQual"].notnull()].groupby(["BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]).agg(lambda x: stats.mode(x)[0][0])["BsmtQual"]

In [ ]:
test["BsmtQual"] = test["BsmtQual"].fillna("TA")

In [ ]:
test[test["BsmtExposure"].isnull()]

In [ ]:
test[test["BsmtExposure"].notnull()].groupby(["BsmtCond","BsmtQual","BsmtFinType1","BsmtFinType2"]).agg(lambda x: stats.mode(x)[0][0])["BsmtExposure"]

In [ ]:
test["BsmtExposure"] = test["BsmtExposure"].fillna("No")

In [ ]:
test["BsmtFinSF1"] = test["BsmtFinSF1"].fillna(test["BsmtFinSF1"].mean())
test["BsmtFinSF2"] = test["BsmtFinSF2"].fillna(test["BsmtFinSF2"].mean())

In [ ]:
test["BsmtUnfSF"] = test["BsmtUnfSF"].fillna(test["BsmtUnfSF"].mean())
test["TotalBsmtSF"] = test["TotalBsmtSF"].fillna(test["TotalBsmtSF"].mean())

Scope of improvement - 


In [ ]:
test["BsmtFullBath"] = test["BsmtFullBath"].fillna(stats.mode(test["BsmtFullBath"])[0][0])
test["BsmtHalfBath"] = test["BsmtHalfBath"].fillna(stats.mode(test["BsmtFullBath"])[0][0])

In [ ]:
test[test["KitchenQual"].isnull()]

In [ ]:
test[test["KitchenQual"].notnull()].groupby(["OverallQual","BsmtQual","GarageQual","ExterQual"]).agg(lambda x: stats.mode(x)[0][0])["KitchenQual"]

In [ ]:
test["KitchenQual"] = test["KitchenQual"].fillna("TA")

In [ ]:
test["Functional"] = test["Functional"].fillna("Typ")

6.**Electrical**

In [ ]:
train[train["Electrical"].isnull()].loc[:,["Heating","HeatingQC","CentralAir"]]

In [ ]:
newDF = train[(train["Electrical"].notnull()) & (train["Heating"]=="GasA") & (train["HeatingQC"]=="Gd") & (train["CentralAir"]=="Y")].groupby(["Heating","HeatingQC","CentralAir"]).agg(lambda x: stats.mode(x)[0][0])
newDF["Electrical"]

In [ ]:
train["Electrical"] = train["Electrical"].fillna("SBrkr")

7.**FireplaceQu**

In [ ]:
train["FireplaceQu"] = train["FireplaceQu"].fillna("NoFireplace") 
test["FireplaceQu"] = test["FireplaceQu"].fillna("NoFireplace")

8.**GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond**

In [ ]:
train.loc[train["GarageType"].isnull(),["GarageType","GarageFinish","GarageQual","GarageCond"]] = "NoGarage"
test.loc[test["GarageType"].isnull(),"GarageYrBlt"] = 2018
test.loc[test["GarageType"].isnull(),["GarageType","GarageFinish","GarageQual","GarageCond"]] = "NoGarage"

In [ ]:
test[test["GarageYrBlt"].isnull()]
test["GarageYrBlt"] = test["GarageYrBlt"].fillna(test.mode()["GarageYrBlt"][0])
test["GarageFinish"] = test["GarageFinish"].fillna(test.mode()["GarageFinish"][0])
test["GarageCars"] = test["GarageCars"].fillna(test.mode()["GarageCars"][0])
test["GarageArea"] = test["GarageArea"].fillna(test["GarageArea"].mean())
test["GarageQual"] = test["GarageQual"].fillna(test.mode()["GarageQual"][0])
test["GarageCond"] = test["GarageCond"].fillna(test.mode()["GarageCond"][0])

**GarageYrBlt - Convert to age, then convert to categorical data and take care of this data.**

9.**PoolQC, Fence, MiscFeature**

In [ ]:
train.loc[train["PoolQC"].isnull(),"PoolQC"] = "NoPool"
train.loc[train["Fence"].isnull(),"Fence"] = "NoFence"
train.loc[train["MiscFeature"].isnull(),"MiscFeature"] = "Nothing"
test.loc[test["PoolQC"].isnull(),"PoolQC"] = "NoPool"
test.loc[test["Fence"].isnull(),"Fence"] = "NoFence"
test.loc[test["MiscFeature"].isnull(),"MiscFeature"] = "Nothing"

In [ ]:
train[nullCols].isnull().sum()

In [ ]:
print(train[train["BsmtExposure"].isnull()])
print(train[train["BsmtFinType2"].isnull()])

In [ ]:
train[train["Id"]==333]

In [ ]:
train[train["BsmtFinType2"].notnull()].groupby(["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1"]).agg(lambda x: stats.mode(x)[0][0])["BsmtFinType2"]

Seeing from the mode of the related features, we can see that the most probable BsmtFinType2 is "Unf".

In [ ]:
train["BsmtFinType2"] = train["BsmtFinType2"].fillna("Unf")

In [ ]:
train[train["Id"]==949]

In [ ]:
train[train["BsmtExposure"].notnull()].groupby(["BsmtQual","BsmtCond","BsmtFinType2","BsmtFinType1"]).agg(lambda x: stats.mode(x)[0][0])["BsmtExposure"]

Following the trend, there is "No" , i.e, No Exposure.

In [ ]:
train["BsmtExposure"] = train["BsmtExposure"].fillna("No")

In [ ]:
train["GarageYrBlt"] = train["GarageYrBlt"].fillna(2018)

In [ ]:
train[nullCols].isnull().sum()

In [ ]:
def convert_year_to_age(year):
    age = 2018 - year
    return age

train["GarageYrBlt"] = train["GarageYrBlt"].apply(convert_year_to_age)
train["YearBuilt"] = train["YearBuilt"].apply(convert_year_to_age)
train["YearRemodAdd"] = train["YearRemodAdd"].apply(convert_year_to_age)

test["GarageYrBlt"] = test["GarageYrBlt"].apply(convert_year_to_age)
test["YearBuilt"] = test["YearBuilt"].apply(convert_year_to_age)
test["YearRemodAdd"] = test["YearRemodAdd"].apply(convert_year_to_age)

In [ ]:
train.YrSold = ((train.MoSold)/12) + train.YrSold
train["YrSold"] = train["YrSold"].apply(convert_year_to_age)

In [ ]:
test.YrSold = ((test.MoSold)/12) + test.YrSold
test["YrSold"] = test["YrSold"].apply(convert_year_to_age)

In [ ]:
train.corr()["LotFrontage"]

Seeing the correlation values, we choose a few features on which the LotFrontage tends to be depending. Those features are :
**MSSubClass
LotArea <br>
TotalBsmtSF<br>
1stFlrSF<br>
GrLivArea<br>
TotRmsAbvGrd<br>**

In [ ]:
trainy = train.copy()

In [ ]:
trainX_for_LotFrontage = trainy[trainy["LotFrontage"].notnull()].loc[:,["MSSubClass","LotArea","TotalBsmtSF","1stFlrSF","GrLivArea","TotRmsAbvGrd"]]
trainY_for_LotFrontage = trainy[trainy["LotFrontage"].notnull()]["LotFrontage"]

In [ ]:
testX_for_LotFrontage = trainy[trainy["LotFrontage"].isnull()].loc[:,["MSSubClass","LotArea","TotalBsmtSF","1stFlrSF","GrLivArea","TotRmsAbvGrd"]]

In [ ]:
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
minMax = MinMaxScaler()
trainX_for_LotFrontage = minMax.fit_transform(trainX_for_LotFrontage)
testX_for_LotFrontage = minMax.fit_transform(testX_for_LotFrontage)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=100)
regr.fit(trainX_for_LotFrontage,trainY_for_LotFrontage)
fillLotFrontage = regr.predict(testX_for_LotFrontage)

In [ ]:
train.loc[train["LotFrontage"].isnull(),"LotFrontage"] = fillLotFrontage

In [ ]:
train[nullCols].isnull().sum()

Thus, all the NaN values have been handled for the training data. Moving forward to handling the Categorical Variables now.

In [ ]:
#CategCols = train.columns[train.dtypes=="object"].tolist()

In [ ]:
#train = pd.get_dummies(train,columns=CategCols)

In [ ]:
train = train.drop(columns="Id")

So our training data is ready...Splitting the data set into 2 parts using the train_test Splitter.

In [ ]:
SalePrice = train["SalePrice"]

In [ ]:
SalePrice = np.log1p(SalePrice)

In [ ]:
#train = train.drop(columns="SalePrice")

In [ ]:
#train["LotFrontage"] = minMax.fit_transform(train["LotFrontage"].values.reshape(-1,1))

In [ ]:
nullCols = test.columns[test.isna().any()].tolist()
test[nullCols].isnull().sum()

In [ ]:
test.corr()["LotFrontage"]

In [ ]:
testy = test.copy()

In [ ]:
trainerX_for_LotFrontage = testy[testy["LotFrontage"].notnull()].loc[:,["MSSubClass","LotArea","GarageArea","1stFlrSF","GrLivArea"]]
trainerY_for_LotFrontage = testy[testy["LotFrontage"].notnull()]["LotFrontage"]

In [ ]:
testerX_for_LotFrontage = testy[testy["LotFrontage"].isnull()].loc[:,["MSSubClass","LotArea","GarageArea","1stFlrSF","GrLivArea"]]

In [ ]:
trainerX_for_LotFrontage = minMax.fit_transform(trainerX_for_LotFrontage)
testerX_for_LotFrontage = minMax.fit_transform(testerX_for_LotFrontage)

In [ ]:
regr.fit(trainerX_for_LotFrontage,trainerY_for_LotFrontage)
fillLotFrontage = regr.predict(testerX_for_LotFrontage)

In [ ]:
test.loc[test["LotFrontage"].isnull(),"LotFrontage"] = fillLotFrontage

In [ ]:
nullCols = test.columns[test.isna().any()].tolist()
test[nullCols].isnull().sum()

In [ ]:
test[test["Exterior1st"].isnull()]

In [ ]:
test.loc[test["Exterior1st"].isnull(),"Exterior1st"] = test[test["Exterior1st"].notnull()].mode()["Exterior1st"][0]
test.loc[test["Exterior2nd"].isnull(),"Exterior2nd"] = test[test["Exterior2nd"].notnull()].mode()["Exterior2nd"][0]

In [ ]:
nullCols = test.columns[test.isna().any()].tolist()
test[nullCols].isnull().sum()

In [ ]:
test[test["Utilities"].isnull()]

In [ ]:
test[test["Utilities"].notnull()].groupby(["Electrical","Heating"]).agg(lambda x: stats.mode(x)[0][0])["Utilities"]

In [ ]:
test["Utilities"] = test["Utilities"].fillna("AllPub")

In [ ]:
test["SaleType"] = test["SaleType"].fillna(test[test["SaleType"].notnull()].mode()["SaleType"][0])

In [ ]:
test[test["MSZoning"].isnull()]

In [ ]:
test[test["MSZoning"].notnull()].groupby("MSSubClass").agg(lambda x: stats.mode(x)[0][0])["MSZoning"]

In [ ]:
test.loc[(test["MSZoning"].isnull() & test["MSSubClass"] == 20),"MSZoning"] = "RL"
test.loc[test["MSZoning"].isnull(),"MSZoning"] = "RM"

In [ ]:
nullCols = test.columns[test.isna().any()].tolist()
test[nullCols].isnull().sum()

In [ ]:
test[test["MasVnrType"].isnull()]

In [ ]:
test[test["MasVnrType"].notnull()].groupby(["MSZoning","MSSubClass","Alley","LotShape","LandContour","LotConfig"]).agg(lambda x: stats.mode(x)[0][0])["MasVnrType"]

In [ ]:
test.loc[test["Id"]==2593, "MasVnrType"] = "BrkFace"

In [ ]:
test["MasVnrType"] = test["MasVnrType"].fillna("None")

In [ ]:
test.loc[test["MasVnrType"]=="None","MasVnrArea"] = 0

In [ ]:
test[test["MasVnrType"]=="BrkFace"].mean()["MasVnrArea"]

In [ ]:
test.loc[test["MasVnrType"]=="BrkFace","MasVnrArea"] = 264.4

In [ ]:
nullCols = test.columns[test.isna().any()].tolist()
test[nullCols].isnull().sum()

In [ ]:
#test["LotFrontage"] = minMax.fit_transform(test["LotFrontage"].values.reshape(-1,1))

In [ ]:
TestCategCols = test.columns[test.dtypes=="object"].tolist()

In [ ]:
#test = pd.get_dummies(test,columns=TestCategCols)

In [ ]:
test = test.drop(columns="Id")

In [ ]:
#FinalPredictedPrice = regr.predict(test)
test.info()

In [ ]:
train = train.drop(columns="MoSold")
test = test.drop(columns ="MoSold")

In [ ]:
train.corr()["SalePrice"]

In [ ]:
train.plot.hexbin(x="YearBuilt",y="SalePrice",gridsize=15,sharex=False)

In [ ]:
def year_built(year):
    age=''
    if year<25:
        age="LessThan25"
    elif year<35:
        age="LessThan35"
    elif year<50:
        age="LessThan50"
    elif year<65:
        age="LessThan65"
    else:
        age="Old"
    return age

train["YearBuilt"] = train["YearBuilt"].apply(year_built)
test["YearBuilt"] = test["YearBuilt"].apply(year_built)

In [ ]:
train.groupby("YearBuilt").mean()["SalePrice"].plot.line()

In [ ]:
train.plot.hexbin(x="YearRemodAdd",y="SalePrice",gridsize=15,sharex=False)

In [ ]:
def year_remod_add(year):
    age=''
    if year<25:
        age="LessThan25"
    elif year<38:
        age="LessThan38"
    elif year<50:
        age="LessThan50"
    elif year<60:
        age="LessThan60"
    else:
        age="Old"
    return age

train["YearRemodAdd"] = train["YearRemodAdd"].apply(year_remod_add)
test["YearRemodAdd"] = test["YearRemodAdd"].apply(year_remod_add)

In [ ]:
train.groupby("YearRemodAdd").mean()["SalePrice"].plot.line()

In [ ]:
train.plot.hexbin(x="YrSold",y="SalePrice",gridsize=15,sharex=False)

In [ ]:
train.plot.hexbin(x="OverallQual",y="SalePrice",gridsize=15,sharex=False)

In [ ]:
train.plot.hexbin(x="OverallCond",y="SalePrice",gridsize=15,sharex=False)

In [ ]:
def overall_cond(score):
    age=''
    if score<4:
        age="LessThanFour"
    elif score<6:
        age="LessThanSix"
    elif score<7:
        age="LessThan7"
    else:
        age="MoreThan8"
    return age

train["OverallCond"] = train["OverallCond"].apply(overall_cond)
test["OverallCond"] = test["OverallCond"].apply(overall_cond)

In [ ]:
train.groupby("OverallCond").mean()["SalePrice"].plot.line()

In [ ]:
train = train.drop(columns="SalePrice")

In [ ]:
train["LotFrontage"] = train["LotFrontage"].apply(lambda x: np.square(x))
test["LotFrontage"] = test["LotFrontage"].apply(lambda x: np.square(x))

In [ ]:
from scipy.stats import norm, skew

In [ ]:
numeric_feats = train.dtypes[train.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

In [ ]:
skewness = skewness[abs(skewness.Skew)>0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    train[feat] = boxcox1p(train[feat], lam)
    test[feat] = boxcox1p(train[feat], lam)

In [ ]:
def QualToInt(x):
    if(x=='Ex'):
        r = 0
    elif(x=='Gd'):
        r = 1
    elif(x=='TA'):
        r = 2
    elif(x=='Fa'):
        r = 3
    elif(x=='missing'):
        r = 4
    else:
        r = 5
    return r

train['ExterQual'] = train['ExterQual'].apply(QualToInt)
train['ExterCond'] = train['ExterCond'].apply(QualToInt)
train['KitchenQual'] = train['KitchenQual'].apply(QualToInt)
train['HeatingQC'] = train['HeatingQC'].apply(QualToInt)
train['BsmtQual'] = train['BsmtQual'].apply(QualToInt)
train['BsmtCond'] = train['BsmtCond'].apply(QualToInt)
train['FireplaceQu'] = train['FireplaceQu'].apply(QualToInt)
train['GarageQual'] = train['GarageQual'].apply(QualToInt)
train['GarageCond'] = train['GarageCond'].apply(QualToInt)
train['PoolQC'] = train['PoolQC'].apply(QualToInt)

test['ExterQual'] = test['ExterQual'].apply(QualToInt)
test['ExterCond'] = test['ExterCond'].apply(QualToInt)
test['KitchenQual'] = test['KitchenQual'].apply(QualToInt)
test['HeatingQC'] = test['HeatingQC'].apply(QualToInt)
test['BsmtQual'] = test['BsmtQual'].apply(QualToInt)
test['BsmtCond'] = test['BsmtCond'].apply(QualToInt)
test['FireplaceQu'] = test['FireplaceQu'].apply(QualToInt)
test['GarageQual'] = test['GarageQual'].apply(QualToInt)
test['GarageCond'] = test['GarageCond'].apply(QualToInt)
test['PoolQC'] = test['PoolQC'].apply(QualToInt)

def SlopeToInt(x):
    if(x=='Gtl'):
        r = 0
    elif(x=='Mod'):
        r = 1
    elif(x=='Sev'):
        r = 2
    else:
        r = 3
    return r

train['LandSlope'] = train['LandSlope'].apply(SlopeToInt)
train['CentralAir'] = train['CentralAir'].apply( lambda x: 0 if x == 'N' else 1) 
train['Street'] = train['Street'].apply( lambda x: 0 if x == 'Pave' else 1) 
train['PavedDrive'] = train['PavedDrive'].apply( lambda x: 0 if x == 'Y' else 1)

test['LandSlope'] = test['LandSlope'].apply(SlopeToInt)
test['CentralAir'] = test['CentralAir'].apply( lambda x: 0 if x == 'N' else 1) 
test['Street'] = test['Street'].apply( lambda x: 0 if x == 'Pave' else 1) 
test['PavedDrive'] = test['PavedDrive'].apply( lambda x: 0 if x == 'Y' else 1)


def GFinishToInt(x):
    if(x=='Fin'):
        r = 0
    elif(x=='RFn'):
        r = 1
    elif(x=='Unf'):
        r = 2
    else:
        r = 3
    return r

train['GarageFinish'] = train['GarageFinish'].apply(GFinishToInt)
test['GarageFinish'] = test['GarageFinish'].apply(GFinishToInt)

def BsmtExposureToInt(x):
    if(x=='Gd'):
        r = 0
    elif(x=='Av'):
        r = 1
    elif(x=='Mn'):
        r = 2
    elif(x=='No'):
        r = 3
    else:
        r = 4
    return r

train['BsmtExposure'] = train['BsmtExposure'].apply(BsmtExposureToInt)
test['BsmtExposure'] = test['BsmtExposure'].apply(BsmtExposureToInt)

def BsmtFinType1ToInt(x):
    if(x=='GLQ'):
        r = 6
    elif(x=='ALQ'):
        r = 5
    elif(x=='BLQ'):
        r = 4
    elif(x=='Rec'):
        r = 3   
    elif(x=='LwQ'):
        r = 2
    elif(x=='Unf'):
        r = 1        
    else:
        r = 0
    return r

train['BsmtFinType1'] = train['BsmtFinType1'].apply(BsmtFinType1ToInt)
test['BsmtFinType1'] = test['BsmtFinType1'].apply(BsmtFinType1ToInt)

train['BsmtFinType2'] = train['BsmtFinType2'].apply(BsmtFinType1ToInt)
test['BsmtFinType2'] = test['BsmtFinType2'].apply(BsmtFinType1ToInt)


In [ ]:
CategCols = train.columns[train.dtypes=="object"].tolist()

In [ ]:
temp = pd.get_dummies(pd.concat([train,test],keys=[0,1]), columns=CategCols)
train,test = temp.xs(0),temp.xs(1)

In [ ]:
#import matplotlib.pyplot as plt
#train[train.dtypes[(train.dtypes=="float64")|(train.dtypes=="int64")].index.values].hist(figsize=[50,50])

In [ ]:
train[train.columns] = minMax.fit_transform(train[train.columns])
test[test.columns] = minMax.fit_transform(test[test.columns])

In [ ]:
#from boruta import BorutaPy
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rfc = RandomForestRegressor(n_estimators=200, n_jobs=4, max_depth=6)
#boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2)
#start_time = timer(None)
#X = train.values
#y = SalePrice.values
#boruta_selector.fit(X, y)
#timer(start_time)

In [ ]:
train = train.drop(['Utilities', 'Street'], axis=1)
test = test.drop(['Utilities', 'Street'], axis=1)

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import GradientBoostingRegressor
GBmodel = GradientBoostingRegressor()
selector = RFECV(GBmodel, step=1, cv=5,n_jobs =4,scoring="neg_mean_squared_log_error",verbose = 2)
start_time = timer(None)
X = train.values
y = SalePrice.values
selector.fit(X, y)
timer(start_time)

In [ ]:
selected = train.columns[selector.support_]
selected

In [ ]:
train = train[selected]
test = test[selected]

In [ ]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components = 30)

#pca.fit_transform(train)
#pca.transform(test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, SalePrice, test_size=0.3, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
regr.fit(X_train, y_train)
PredictedPrice = regr.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.linear_model import Ridge
RidgeReg = Ridge()
RidgeReg.fit(X_train,y_train)
#PredictedPrice = np.absolute(lassoReg.predict(X_test))
PredictedPrice = np.absolute(RidgeReg.predict(X_test))
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.linear_model import Lasso
RidgeReg = Lasso()
RidgeReg.fit(X_train,y_train)
PredictedPrice = np.absolute(RidgeReg.predict(X_test))
#PredictedPrice = RidgeReg.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
import xgboost as xgb
modelxgb = xgb.XGBRegressor()
modelxgb.fit(X_train,y_train)
PredictedPrice = np.absolute(modelxgb.predict(X_test))
#PredictedPrice = RidgeReg.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(X_train,y_train)
PredictedPrice = np.absolute(model.predict(X_test))
#PredictedPrice = RidgeReg.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.svm import SVR
SVmodel = SVR()
SVmodel.fit(X_train,y_train)
PredictedPrice = np.absolute(SVmodel.predict(X_test))
#PredictedPrice = RidgeReg.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.linear_model import ElasticNet
model = ElasticNet()
model.fit(X_train,y_train)
PredictedPrice = np.absolute(model.predict(X_test))
#PredictedPrice = RidgeReg.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.linear_model import ElasticNetCV
ELmodel = ElasticNetCV(cv=5, random_state=0)
ELmodel.fit(X_train,y_train)
PredictedPrice = ELmodel.predict(X_test)
#PredictedPrice = RidgeReg.predict(X_test)
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train,y_train)
PredictedPrice = np.absolute(GBmodel.predict(X_test))
y_test1 = np.log(y_test)
#PredictedPrice = np.log(PredictedPrice)
mean_squared_error(y_test,PredictedPrice)

In [ ]:
#from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.model_selection import GridSearchCV
#gdb = GradientBoostingRegressor()
#parameter_grid = {
#             'max_depth' : [4, 6, 8],
#             'n_estimators': [10, 50,100],
#             'max_features': ['sqrt', 'auto', 'log2'],
#             'min_samples_split': [0.001,0.003,0.01],
#             'min_samples_leaf': [1, 3, 10]
#             }
#model = GridSearchCV(gdb,scoring="neg_mean_squared_log_error",param_grid=parameter_grid,cv=5)
#model.fit(X_train,y_train)
#PredictedPrice = np.absolute(model.predict(X_test))
#y_test1 = np.log(y_test)
##PredictedPrice = np.log(PredictedPrice)
#mean_squared_error(y_test,PredictedPrice)

In [ ]:
testY = pd.read_csv("../input/test.csv")
FinalPredictedPrice = np.expm1(RidgeReg.predict(test))
Submission = pd.DataFrame({'Id':testY.Id,'SalePrice':FinalPredictedPrice})
Submission.to_csv('submission.csv', index=False)